In [13]:
%matplotlib inline 
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt

########
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

warnings.filterwarnings('ignore')
plt.style.use('seaborn-whitegrid')

In [14]:
LA_data_df = pd.read_csv('LA.csv', na_values=[' ','Nan','nan',''])

In [16]:
LA_data_df.PIPE_MAT.value_counts()

VCP     133208
CON      14900
UNK       5816
RCP       2043
PVC       1270
CIP        910
B/C        775
DIP        579
HDPE       227
ABS        225
GFR        112
STL         98
ACP         53
PCP         33
PEP         15
CMP          6
TRS          5
CIPC         4
TCP          2
RCPL         1
Name: PIPE_MAT, dtype: int64

In [25]:
mask = "ABS"
LA_data_df.MATERIAL_DESCRIPTION[LA_data_df['PIPE_MAT'] == mask]

469       Acrylonitrile Butadiene Styrene
1000      Acrylonitrile Butadiene Styrene
1228      Acrylonitrile Butadiene Styrene
2717      Acrylonitrile Butadiene Styrene
3427      Acrylonitrile Butadiene Styrene
4240      Acrylonitrile Butadiene Styrene
4298      Acrylonitrile Butadiene Styrene
4665      Acrylonitrile Butadiene Styrene
7221      Acrylonitrile Butadiene Styrene
8980      Acrylonitrile Butadiene Styrene
9574      Acrylonitrile Butadiene Styrene
11944     Acrylonitrile Butadiene Styrene
12598     Acrylonitrile Butadiene Styrene
13532     Acrylonitrile Butadiene Styrene
16554     Acrylonitrile Butadiene Styrene
17030     Acrylonitrile Butadiene Styrene
17039     Acrylonitrile Butadiene Styrene
18661     Acrylonitrile Butadiene Styrene
22735     Acrylonitrile Butadiene Styrene
25517     Acrylonitrile Butadiene Styrene
25518     Acrylonitrile Butadiene Styrene
26510     Acrylonitrile Butadiene Styrene
26907     Acrylonitrile Butadiene Styrene
26909     Acrylonitrile Butadiene 

In [2]:
LA_data_df.drop(['MON_INST','REHAB_DESC', 'MATERIAL_DESCRIPTION'],axis=1, inplace=True)
LA_data_df.sample(10)

,CLASS,PIPE_LENG,SLOPE_PCT,PIPE_SIZE,PIPE_WIDTH,PIPE_MAT,PIPE_SHAPE,REHAB,PIPE_STATUS,SHAPE_Length
109180,3,234.1,0.0760,8.0,0.0,VCP,CR,N,F,82.194
110536,3,350.0,0.0040,8.0,0.0,VCP,CIR,N,F,128.675
19737,3,279.8,0.0480,8.0,0.0,VCP,CIR,N,F,103.064
71517,3,390.0,0.0276,8.0,0.0,PVC,CIR,N,F,144.290
48688,3,274.2,0.0080,10.0,0.0,VCP,CIR,N,F,101.150
27450,3,152.9,0.0040,8.0,0.0,VCP,CR,N,F,56.445
71251,3,330.8,0.0060,8.0,0.0,VCP,CR,N,F,122.145
135164,3,241.0,0.0340,8.0,0.0,VCP,CIR,Y,F,88.823
79875,3,339.4,0.0072,8.0,0.0,VCP,CR,N,F,125.288
50895,1,350.4,0.0030,52.0,0.0,CON,BM,N,F,131.657


In [19]:
LA_data_df.isnull().sum()

CLASS                       0
PIPE_LENG                  20
SLOPE_PCT                 485
PIPE_SIZE                  34
PIPE_WIDTH                551
PIPE_MAT                   88
PIPE_SHAPE               1270
REHAB                     647
PIPE_STATUS                 8
MON_INST                54740
SHAPE_Length                0
MATERIAL_DESCRIPTION      482
REHAB_DESC                646
dtype: int64

In [20]:
LA_data_df.shape

(160370, 13)

In [21]:
LA_data_df = LA_data_df.dropna(axis=0)
LA_data_df.sample(10)

,CLASS,PIPE_LENG,SLOPE_PCT,PIPE_SIZE,PIPE_WIDTH,PIPE_MAT,PIPE_SHAPE,REHAB,PIPE_STATUS,MON_INST,SHAPE_Length,MATERIAL_DESCRIPTION,REHAB_DESC
96387,3,278.00,0.0040,8.0,0.0,VCP,CR,N,F,0,102.137,Vitrified Clay Pipe,No
16669,3,113.70,0.0200,8.0,0.0,VCP,CIR,N,F,1,41.950,Vitrified Clay Pipe,No
63110,3,222.00,0.0652,8.0,0.0,VCP,CR,N,F,7,81.602,Vitrified Clay Pipe,No
98636,3,120.30,0.0060,8.0,0.0,VCP,CIR,N,F,0,44.339,Vitrified Clay Pipe,No
31653,3,247.18,0.0079,8.0,0.0,VCP,CIR,N,F,7,90.770,Vitrified Clay Pipe,No
28557,3,271.60,0.1300,8.0,0.0,VCP,CIR,Y,F,9,100.110,Vitrified Clay Pipe,Yes
7538,3,260.00,0.0340,8.0,0.0,VCP,CR,N,F,0,96.038,Vitrified Clay Pipe,No
84235,3,350.00,0.0068,8.0,0.0,VCP,CR,N,F,3,129.004,Vitrified Clay Pipe,No
90645,3,300.20,0.0050,8.0,0.0,VCP,CR,N,F,1,110.273,Vitrified Clay Pipe,No
58101,3,300.00,0.0111,8.0,0.0,VCP,CIR,N,F,0,111.022,Vitrified Clay Pipe,No


In [6]:
LA_data_df.shape

(157974, 10)

In [22]:
LA_data_df.PIPE_MAT.value_counts()

VCP    87280
CON    10708
RCP     1972
UNK     1076
PVC     1042
CIP      766
B/C      737
DIP      504
ABS      193
STL       87
ACP       44
CMP        5
TRS        1
Name: PIPE_MAT, dtype: int64

In [7]:
LA_data_df.isnull().sum()

CLASS           0
PIPE_LENG       0
SLOPE_PCT       0
PIPE_SIZE       0
PIPE_WIDTH      0
PIPE_MAT        0
PIPE_SHAPE      0
REHAB           0
PIPE_STATUS     0
SHAPE_Length    0
dtype: int64

,CLASS,PIPE_LENG,SLOPE_PCT,PIPE_SIZE,PIPE_WIDTH,PIPE_MAT,PIPE_SHAPE,REHAB,PIPE_STATUS,SHAPE_Length
5,2,54.40,0.0009,20.0,0.0,CIP,CIR,N,F,20.154
13,2,228.20,0.0012,20.0,0.0,CIP,CR,N,F,82.451
80,3,64.10,0.0000,8.0,0.0,CIP,CR,N,F,23.473
183,3,-99.00,-99.0000,8.0,0.0,CIP,CR,N,N,4.058
515,3,278.40,0.0121,8.0,0.0,CIP,CR,N,F,102.221
524,3,89.20,0.0044,8.0,0.0,CIP,CR,N,F,32.832
762,2,20.00,0.0020,12.0,0.0,CIP,CIR,Y,S,7.887
1119,3,796.10,0.1248,6.0,0.0,CIP,CIR,N,F,293.191
1204,3,80.00,0.0212,8.0,0.0,CIP,CIR,N,F,30.678
1332,3,111.00,0.0713,8.0,0.0,CIP,CR,N,F,49.769


In [ ]:
160370 - 157602 

In [ ]:
LA_data_df.info()

In [ ]:
LA_data_df.describe()

In [ ]:
LA_data_df['PIPE_SHAPE'].value_counts()

In [ ]:
LA_data_df = pd.get_dummies(LA_data_df, columns=['PIPE_SHAPE'])
LA_data_df.head()

In [ ]:
LA_data_df = pd.get_dummies(LA_data_df, columns=['PIPE_MAT'])
LA_data_df.head()

In [ ]:
LA_data_df = pd.get_dummies(LA_data_df, columns=['PIPE_STATUS'])
LA_data_df.head()

In [ ]:
#convert target into numerical value
LA_data_df['REHAB'] = LA_data_df.REHAB.astype("category").cat.codes

In [ ]:
y = LA_data_df['REHAB']
X = LA_data_df.drop(['REHAB'], axis=1)

In [ ]:
y.shape

In [ ]:
X.shape

In [ ]:
#MLP

'''
from sklearn.manifold import Isomap
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3 , random_state=0)
model = MLPClassifier(solver='lbfgs', activation='relu', random_state=0,hidden_layer_sizes=[5,2])    
model.fit(X_train,y_train)

y_train_hat = model.predict(X_train) #in-sample
y_test_hat = model.predict(X_test) #out-of-sample     
                      
in_sample_acc = accuracy_score(y_train, y_train_hat)
out_of_sample_acc = accuracy_score(y_test, y_test_hat)

print("Accuracy of in_sample data {}".format(in_sample_acc))
print("Accuracy of Out_of_sample data {}".format(out_of_sample_acc));
'''

In [ ]:
# LogistRegression
'''
from sklearn.linear_model import LogisticRegression
LogR = LogisticRegression(solver='lbfgs', random_state=0)
LogR.fit(X_train, y_train)

y_train_hat = LogR.predict(X_train)
y_test_hat = LogR.predict(X_test)

in_sample_acc = accuracy_score(y_train, y_train_hat)
out_of_sample_acc = accuracy_score(y_test, y_test_hat)

print("Accuracy of in_sample data {}".format(in_sample_acc))
print("Accuracy of Out_of_sample data {}".format(out_of_sample_acc));
'''

In [ ]:
# random forest 
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3 , random_state=0)

#initialize
RFC = RandomForestClassifier(n_estimators=100)
RFC.fit(X_train, y_train)

y_train_hat = RFC.predict(X_train)
y_test_hat = RFC.predict(X_test) #out-of-sample

in_sample_acc = accuracy_score(y_train, y_train_hat)
out_of_sample_acc = accuracy_score(y_test, y_test_hat)

print("Accuracy of in_sample data {}".format(in_sample_acc))
print("Accuracy of Out_of_sample data {}".format(out_of_sample_acc));

<b>Question:</b> What if we want to get all the measures? Newer version of scikit-learn allows you to compute the multiple scores for the cross validation using <b style='color:blue'>cross_validate</b> function

<p><b>Note:</b> You need to use <b style='color:blue'>StratifiedKFold( )</b> to form your desired number of stratified partitions and then pass that to the <b>cv</b> parameter inside the <b style='color:blue'>cross_val_score( )</b> function.</p>

<p><b>Example:</b></p>

In [ ]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
accuracy_rfc = cross_val_score(RFC, X, y, cv=cv, scoring='accuracy')

print("Cross Validation Scores for Random Forest: {}".format(accuracy_rfc))
print("Mean Accuracy Score for Random Forest: {}".format(np.mean(accuracy_rfc)))

### 1.2. Grid Search with Cross-Validation

<p>In some cases deciding about the hyper-parameters becomes very critical point in formation of your model. For example, when you are using KNN to form your predictive model, the selection of right number of neighbors is crucial.  One simple approach is to run your model with different number of neighbors and compare the accuracy score across models. However, this approach becomes really inefficient if you have large set of hyperparameters to test.</p>

<p>Grid search with cross validation allows you to specify different hyperparameters' values and test the performance of each separately using cross validation method. Let's see an example for KNN model.</p>

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [10,20,30,40,50,100]}
cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
grid = GridSearchCV(RFC, param_grid, cv = cv, scoring='accuracy',
                    return_train_score=True)
grid.fit(X, y)

print("Best Parameter: {}".format(grid.best_params_))
print("Best Cross Vlidation Score: {}".format(grid.best_score_))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_test_hat))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_test_hat)

In [ ]:
y_test.unique()

In [ ]:
 RFC.predict_proba(X_test)

<p><b>Question:</b> Run the same model using linear regression and compare the result.</p>

## 3. Receiver operating characteristics (ROC) and AUC

In last session, we become familiar with measurements such as precision, recall and F1 which allow us to capture the performance of the trained model. We also used the precion-recall curve to measure the performance of the model using different threshold values.

There is another commonly used tool named Receiver operating characteristics curve or ROC curve in short, that allows us to consider all possible thresholds for a given classifier. But instead of reporting precision and recall, it shows the false positive rate (FPR) against the true positive rate (TPR). Recall that the true positive rate is simply another name for recall, while the false positive rate is the fraction of false positives out of all negative samples. 

<b>Example:</b> Let's consider the Wisconsin Breast Cancer Example once again 

<b>Note:</b> You can use <b style='color:blue'>roc_curve</b> function in sklearn package to form ROC curve

In [ ]:
from sklearn.metrics import roc_curve

probs = RFC.predict_proba(X_test)
preds = probs[:,1]

fpr, tpr, thresholds = roc_curve(y_test, preds)

plt.plot(fpr, tpr, label="ROC Curve")
plt.xlabel("FPR")
plt.ylabel("TPR (recall)");

In [ ]:
import sklearn.metrics as metrics

roc_auc = metrics.auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#thresholds

In [ ]:
#fpr

In [ ]:
#tpr

For the ROC curve, the ideal curve is close to the top left: you want a classifier that produces a high recall while keeping a low false positive rate.

<b>Note:</b> The point (threshold) closest to the top left might be a better operating point than the one chosen by default (.5). <b>Be aware</b> that choosing a threshold should not be done on the test set, but on a separate <u>validation set</u>.

<b>Question:</b> What is the validation set? why we should use it?

<b>Note:</b> We often want to summarize the ROC curve using a single number, the area under the curve (this is commonly just referred to as the AUC). We can compute the area under the ROC curve using the <b style="color:blue">roc_auc_score</b> function

In [ ]:
from sklearn.metrics import roc_auc_score

RFC_auc = roc_auc_score(y_test, RFC.predict_proba(X_test)[:,1]) 
print("AUC for RFC: {:.3f}".format(RFC_auc))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# define the keras model
model = Sequential()
model.add(Dense(4, input_dim=43, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=2, batch_size=10)


# evaluate the keras model
#accuracy = model.evaluate(X_test, y_test)
#print("Accuracy: {}".format(accuracy*100))

In [ ]:
choriom zadi too topoli be halate istade